In [48]:
"""
import csv
import cv2
import numpy as np


lines = []
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

lines.pop(0)

images = []
measurements = []
for line in lines:
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = './data/IMG/' + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
    
augmented_images = []
augmented_measurements = []
for image, measurement in zip(images,measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurement*-1.0)
    
X_data = np.array(augmented_images)
Y_data = np.array(augmented_measurements)
"""

"\nimport csv\nimport cv2\nimport numpy as np\n\n\nlines = []\nwith open('./data/driving_log.csv') as csvfile:\n    reader = csv.reader(csvfile)\n    for line in reader:\n        lines.append(line)\n\nlines.pop(0)\n\nimages = []\nmeasurements = []\nfor line in lines:\n    source_path = line[0]\n    filename = source_path.split('/')[-1]\n    current_path = './data/IMG/' + filename\n    image = cv2.imread(current_path)\n    images.append(image)\n    measurement = float(line[3])\n    measurements.append(measurement)\n    \naugmented_images = []\naugmented_measurements = []\nfor image, measurement in zip(images,measurements):\n    augmented_images.append(image)\n    augmented_measurements.append(measurement)\n    augmented_images.append(cv2.flip(image,1))\n    augmented_measurements.append(measurement*-1.0)\n    \nX_data = np.array(augmented_images)\nY_data = np.array(augmented_measurements)\n"

In [61]:
import sklearn
import csv
import cv2
import numpy as np


samples = []
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

samples.pop(0)

def generator(samples, batch_size=64):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images = []
            angles = []
            for batch_sample in batch_samples:
                name = './data/IMG/'+batch_sample[0].split('/')[-1]
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)
                images.append(cv2.flip(image,1))
                angles.append(measurement*-1.0)

            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

In [62]:
from sklearn.model_selection import train_test_split

train_samples, valid_samples = train_test_split(samples, test_size=0.2)
train_generator = generator(train_samples, batch_size=256)
valid_generator = generator(valid_samples, batch_size=256)

In [63]:
#import matplotlib.pyplot as plt
#plt.imshow(images[0])
#print(np.shape(X_train[0]))

In [64]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Flatten())
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
#model.fit(X_train, y_train, validation_split=0.2, shuffle=True)
model.fit_generator(train_generator, samples_per_epoch=len(train_samples), validation_data=valid_generator, 
                    nb_val_samples=len(valid_samples), nb_epoch=3)

model.save('model.h5')

Epoch 1/3
6144/6428 [===========================>..] - ETA: 0s - loss: 3.7364

C:\Users\User\Anaconda3\envs\carnd-term1\lib\site-packages\keras\engine\training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


6656/6428 [===============================] - 31s - loss: 3.6478 - val_loss: 0.8089
Epoch 2/3
6712/6428 [===============================] - 21s - loss: 0.7207 - val_loss: 0.1722
Epoch 3/3
6656/6428 [===============================] - 20s - loss: 0.1947 - val_loss: 0.0843
